In [ ]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest' 
#Original Sandbox Environment: 'https://sandbox-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'15',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '0ad53085-1cb2-4eb8-ad9e-3ffbd7e56509',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  #print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)





In [ ]:
import pandas as pd

#viewing data above the limit
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)



In [ ]:
#Normalizing the data to makes it all pretty in a dataframe

df = pd.json_normalize(data['data'])
df['timestamp'] = pd.to_datetime('now')
df


In [ ]:
#defining API runner and keeping in a data frame
def api_runner():
    global df
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest' 
    #Original Sandbox Environment: 'https://sandbox-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
      'start':'1',
      'limit':'15',
      'convert':'USD'
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': '0ad53085-1cb2-4eb8-ad9e-3ffbd7e56509',
    }

    session = Session()
    session.headers.update(headers)

    try:
      response = session.get(url, params=parameters)
      data = json.loads(response.text)
      #print(data)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
      print(e)

    
    #keeping it in a dataframe;
    
    #df2 = pd.json_normalize(data['data'])
    #df2['Timestamp'] = pd.to_datetime('now')
    #df = df.append(df2)


    #creating a csv and append data to it
    
    df = pd.json_normalize(data['data'])
    df['timestamp'] = pd.to_datetime('now')
    df

    if not os.path.isfile(r'C:\Users\HP\Documents\API.csv'):
        df.to_csv(r'C:\Users\HP\Documents\API.csv', header = 'column_names')
    else:
        df.to_csv(r'C:\Users\HP\Documents\API.csv', mode = 'a' , header = False)
        


In [ ]:
import os 
from time import time
from time import sleep

for i in range(333):
    api_runner()
    print('API Runner completed')
    sleep(60) #sleep for 1 minute
exit()



In [ ]:
df72 = pd.read_csv(r'C:\Users\HP\Documents\API.csv')
df72

df



In [ ]:
# To view the numbers properly using lambda notation

pd.set_option('display.float_format', lambda x: '%.5f' % x)

df



In [ ]:
# Looking at the coin trends over time

df3 = df.groupby('name', sort=False)[['quote.USD.percent_change_1h','quote.USD.percent_change_24h','quote.USD.percent_change_7d','quote.USD.percent_change_30d','quote.USD.percent_change_60d','quote.USD.percent_change_90d']].mean()
df3

df4 = df3.stack()
df4


In [ ]:

#converting the stack to a dataframe
df5 = df4.to_frame(name='values')
df5



In [ ]:
# preview number of items to set index
df5.count()


In [ ]:

#creating a range and pass that as the dataframe
index = pd.Index(range(90))



In [ ]:
# Set the above DataFrame index object as the index
# using reset_index() function
df6 = df5.reset_index()
df6



In [ ]:
# Changing the column name
df7 = df6.rename(columns={'level_1': 'percent_change'})
df7

df7['percent_change'] = df7['percent_change'].replace(['quote.USD.percent_change_1h', 'quote.USD.percent_change_24h','quote.USD.percent_change_7d','quote.USD.percent_change_30d','quote.USD.percent_change_60d','quote.USD.percent_change_90d'],['1h','24h','7d','30d','60d','90d'])
df7



In [ ]:
#Creating a visualization
import seaborn as sns
import matplotlib.pyplot as plt

sns.catplot(x='percent_change', y='values', hue='name', data=df7, kind='point')



In [ ]:
# Creating a dataframe with other colums
df10 = df[['name','quote.USD.price','timestamp']]
df10 = df10.query("name == 'Bitcoin'")
df10

In [ ]:
#Visualizing
sns.set_theme(style="darkgrid")

sns.lineplot(x='timestamp', y='quote.USD.price', data = df10)
